In [1]:
# %matplotlib inline
%matplotlib ipympl

from IPython.display import clear_output

from datetime import datetime
import os
from os.path import dirname, abspath, join
from os import getcwd
import sys

import random
import string
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D


import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def f(x, y):
    return np.sin(np.sqrt(x ** 2 + y ** 2))

In [3]:
x = np.linspace(-1, 1, 30)
y = np.linspace(-1, 1, 30)

X, Y = np.meshgrid(x, y)
Z = f(X, Y)

In [4]:
fig = plt.figure(figsize=(5,5))
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')
ax.set_title('surface');

FigureCanvasNbAgg()

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

#Class definitions for NN model and learning algorithm
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(N_STATES, HIDDEN_LAYER)
        nn.init.kaiming_uniform_(self.fc1.weight)
        
#         self.fc2 = nn.Linear(HIDDEN_LAYER, HIDDEN_LAYER)
#         nn.init.kaiming_uniform_(self.fc2.weight)
        
#         self.fc3 = nn.Linear(HIDDEN_LAYER, HIDDEN_LAYER)
#         nn.init.kaiming_uniform_(self.fc3.weight)

        self.fc_out = nn.Linear(HIDDEN_LAYER, N_ACTIONS)
        nn.init.xavier_uniform_(self.fc_out.weight) 
    
    def forward(self, x):
        x = self.fc1(x)
        if   (ACTIVATION==0): x = F.relu(x)
        elif (ACTIVATION==1): x = F.leaky_relu(x)
        else                : print("NN ACTIVATION: Error")
#         x = F.leaky_relu(x)
#         x = self.fc2(x)
#         x = F.relu(x)
#         x = self.fc3(x)
#         x = F.relu(x)
        x = self.fc_out(x)
        return x
    
class DQN(object):
    def __init__(self):
        if(GPU): 
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        else:
            device = torch.device('cpu')
        self.eval_net = Net()
        self.eval_net.to(device)
        self.device = device
        print("Neural net:")
        print(self.eval_net)

#         if   (LOSS_F==0): self.loss_func = nn.MSELoss()
#         elif (LOSS_F==1): self.loss_func = nn.SmoothL1Loss()
#         else                : print("LOSS FUNCTION: Error")    
  
    def choose_greedy_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        x = x.to(self.device)

        # input only one sample
        if True:   # greedy
            actions_value = self.eval_net.forward(x)
            actions_value = actions_value.to(torch.device("cpu"))
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] # return the argmax index
        return action

In [16]:
# Hyper Parameters
N_ACTIONS           = 10           #no. of duty cycles (0 to 9)
N_STATES            = 4            #number of state space parameter [batt, enp, henergy, fcast]
GPU                 = False
ACTIVATION          = 0            # 0 = RELU, 1 = Leaky RELU
HIDDEN_LAYER        = 50

In [13]:
NAME = "I1"
seed_arg = 4

In [14]:
seedlist = np.array([161, 314, 228, 271828, 230, 4271031, 5526538, 6610165, 9849252, 34534, 73422, 8765])
seed = seedlist[seed_arg]
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [15]:
MODELNAME  = NAME + '_' + str(seed) + '.pt'
print("MODEL NAME: ", MODELNAME)
S_FILENAME = './models/'+ MODELNAME

MODEL NAME:  I1_230.pt


In [17]:
dqn = DQN()
dqn.eval_net.load_state_dict(torch.load(S_FILENAME))
dqn.eval_net.eval()

Neural net:
Net(
  (fc1): Linear(in_features=4, out_features=50, bias=True)
  (fc_out): Linear(in_features=50, out_features=10, bias=True)
)


Net(
  (fc1): Linear(in_features=4, out_features=50, bias=True)
  (fc_out): Linear(in_features=50, out_features=10, bias=True)
)